In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf

CNN_PATH = "ClimSim/baseline_models/CNN/model/"
MLP_PATH = "ClimSim/baseline_models/MLP/model/backup_phase-7_retrained_models_step2_lot-147_trial_0027.best.h5"

# cnn_model = tf.keras.layers.TFSMLayer(CNN_PATH, call_endpoint="serving_default")
mlp_model = tf.keras.models.load_model(MLP_PATH, compile=False)
mlp_model.summary()

2026-01-28 11:07:36.239467: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-28 11:07:36.682090: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-28 11:07:36.684281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 11:07:37.926526: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Model: "retrained_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 124)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 768)                  96000     ['input[0][0]']               
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 768)                  0         ['dense[0][0]']               
                                                                                                  
 dense_1 (Dense)             (None, 640)                  492160    ['leaky_re_lu[0][0]']         
                                                                                    

2026-01-28 11:07:38.906249: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2026-01-28 11:07:38.906275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: alexandre-tonon-XPS-15-9570
2026-01-28 11:07:38.906281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: alexandre-tonon-XPS-15-9570
2026-01-28 11:07:38.906329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.274.2
2026-01-28 11:07:38.906348: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.274.2
2026-01-28 11:07:38.906354: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 535.274.2


In [2]:
import xarray as xr
import numpy as np
import torch 
from torch.utils.data import Dataset, Subset
import re


class ClimSimBase:
    def __init__(self, zarr_path, grid_path, norm_path, features, normalize=True):
        self.ds = xr.open_zarr(zarr_path, chunks="auto")
        self.features = features
        self.normalize_flag = normalize
        
        self.setup_tendencies()
        self.features_list = self.__get_features__()
        self.ds = self.ds[self.features_list]

        self.grid = xr.open_dataset(grid_path)
        
        self.input_mean = xr.open_dataset(f"{norm_path}inputs/input_mean.nc")
        self.input_std = xr.open_dataset(f"{norm_path}inputs/input_std.nc")
        self.output_scale = xr.open_dataset(f"{norm_path}outputs/output_scale.nc")

        self.input_vars = [v for v in self.features_list if 'in' in v]
        self.output_vars = [v for v in self.features_list if 'out' in v]

    def setup_tendencies(self):    
        timestep = 1200 
        self.ds['out_ptend_t'] = (self.ds['out_state_t'] - self.ds['in_state_t']) / timestep
        self.ds['out_ptend_q0001'] = (self.ds['out_state_q0001'] - self.ds['in_state_q0001']) / timestep
        self.ds['out_ptend_u'] = (self.ds['out_state_u'] - self.ds['in_state_u']) / timestep
        self.ds['out_ptend_v'] = (self.ds['out_state_v'] - self.ds['in_state_v']) / timestep

    def __get_features__(self):
        feat = np.concatenate([self.features["features"]["tendancies"], self.features["features"]["surface"]])
        target = np.concatenate([self.features["target"]["tendancies"], self.features["target"]["surface"]])
        return np.concatenate([feat, target])

    def _prepare_data(self, idx):
        """La méthode 'cœur' qui extrait et formate"""
        def process_list(vars_list, is_input=True):
            out_list = []
            for var in vars_list:
                data = self.ds[var][idx].values
                short_name = re.sub(r'^(in_|out_)', '', var)
                
                if self.normalize_flag:
                    if is_input:
                        data = (data - self.input_mean[short_name].values) / (self.input_std[short_name].values + 1e-8)
                    else:
                        data = data * self.output_scale[short_name].values

                # Gestion 1D (surface) vs 3D (lev)
                data = data[:, np.newaxis] if data.ndim == 1 else data.T
                out_list.append(data)
            return np.concatenate(out_list, axis=1).astype(np.float32)

        return process_list(self.input_vars, True), process_list(self.output_vars, False)

    def __len__(self):
        return self.ds.dims['sample']
    

class ClimSimPyTorch(ClimSimBase, Dataset):
    def __getitem__(self, idx):
        x_np, y_np = self._prepare_data(idx)
        return torch.from_numpy(x_np), torch.from_numpy(y_np)

    # On peut remettre ta méthode de split ici
    def train_test_split(self, test_size=0.2, seed=42, shuffle=True):
        n = len(self)
        indices = np.arange(n)
        if shuffle:
            rng = np.random.default_rng(seed)
            rng.shuffle(indices)
        split = int((1 - test_size) * n)
        return Subset(self, indices[:split]), Subset(self, indices[split:])
    
    def get_models_dims(self, variables_dict):
        features_tend = variables_dict["features"]["tendancies"]
        features_surf = variables_dict["features"]["surface"]
        
        target_tend = variables_dict["target"]["tendancies"]
        target_surf = variables_dict["target"]["surface"]

        def get_var_dim(var):
            if 'lev' in self.ds[var].dims:
                return self.ds[var].sizes['lev']
            return 1

        in_tend_dim = sum([get_var_dim(var) for var in features_tend])
        in_surf_dim = len(features_surf)
        
        out_tend_dim = sum([get_var_dim(var) for var in target_tend])
        out_surf_dim = len(target_surf)

        return {
            "input_total": in_tend_dim + in_surf_dim,
            "output_tendancies": out_tend_dim,
            "output_surface": out_surf_dim
        }
    
class ClimSimKeras(ClimSimBase):
    def get_batch_for_keras(self, idx):
        x_np, y_np = self._prepare_data(idx)
        # Ajout de la dimension Batch (1, ncol, n_feat)
        return np.expand_dims(x_np, axis=0), np.expand_dims(y_np, axis=0)

In [5]:
ZARR_PATH = "/media/alexandre-tonon/UBUNTU 24_0/articleDL/data/ClimSim_low-res.zarr"
GRID_PATH = "data/ClimSim_low-res/ClimSim_low-res_grid-info.nc"
NORM_PATH = "ClimSim/preprocessing/normalizations"

FEATURES = {
    "features" :{
        "tendancies" : ["in_state_t", "in_state_q0001", "in_state_ps"],
        "surface" : ["in_pbuf_LHFLX", "in_pbuf_SHFLX", "in_pbuf_SOLIN"],
    },  
    "target" :{
        "tendancies" : ["out_ptend_t", "out_ptend_q0001"],
        "surface" : ["out_cam_out_NETSW", "out_cam_out_FLWDS", "out_cam_out_PRECSC", "out_cam_out_PRECC", "out_cam_out_SOLS", "out_cam_out_SOLL", "out_cam_out_SOLSD", "out_cam_out_SOLLD"]
    }
}


ClimSimKeras(ZARR_PATH, GRID_PATH, NORM_PATH, FEATURES)

FileNotFoundError: No such file or directory: '/media/alexandre-tonon/UBUNTU 24_0/articleDL/data/ClimSim_low-res.zarr'